In [1]:
import numpy as np
import pandas as pd

# 모델 관련 라이브러리
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [3]:
def preprocessing(train, test):
    
    # 1. 필요 없는 컬럼 제거
    non_imp_cols = ['난자 혼합 경과일',
    '기증자 정자와 혼합된 난자 수',
    '동결 배아 사용 여부',
    '여성 주 불임 원인',
    '불명확 불임 원인',
    '부부 부 불임 원인',
    '착상 전 유전 진단 사용 여부',
    '시술 유형',
    '남성 부 불임 원인',
    '기증 배아 사용 여부',
    '불임 원인 - 정자 운동성',
    '남성 주 불임 원인',
    '불임 원인 - 자궁내막증',
    '불임 원인 - 정자 농도',
    '여성 부 불임 원인',
    '대리모 여부',
    '불임 원인 - 정자 면역학적 요인',
    '저장된 신선 난자 수',
    '부부 주 불임 원인',
    '착상 전 유전 검사 사용 여부',
    '불임 원인 - 여성 요인',
    '불임 원인 - 자궁경부 문제',
    'PGD 시술 여부',
    'PGS 시술 여부',
    '난자 채취 경과일',
    '난자 해동 경과일',
    '불임 원인 - 정자 형태']
    # '착상 전 유전 검사 사용 여부',
    # '착상 전 유전 진단 사용 여부',
    # '남성 주 불임 원인',
    # '남성 부 불임 원인',
    # '여성 주 불임 원인',
    # '여성 부 불임 원인',
    # '부부 주 불임 원인',
    # '부부 부 불임 원인',
    # '불명확 불임 원인',
    # '불임 원인 - 난관 질환',
    # '불임 원인 - 남성 요인',
    # '불임 원인 - 배란 장애',
    # '불임 원인 - 여성 요인',
    # '불임 원인 - 자궁경부 문제',
    # '불임 원인 - 자궁내막증',
    # '불임 원인 - 정자 농도',
    # '불임 원인 - 정자 면역학적 요인',
    # '불임 원인 - 정자 운동성',
    # '불임 원인 - 정자 형태',
    # 'IVF 시술 횟수',
    # 'DI 시술 횟수',
    # '총 임신 횟수',
    # 'IVF 임신 횟수',
    # 'DI 임신 횟수',
    # '총 출산 횟수',
    # 'IVF 출산 횟수',
    # 'DI 출산 횟수',
    # 'PGD 시술 여부',
    # 'PGS 시술 여부',
    # '대리모 여부',
    # '난자 채취 경과일',
    # '배란 유도 유형',
    # '정자 기증자 나이']
    
    train.drop(columns=non_imp_cols, inplace=True)
    test.drop(columns=non_imp_cols, inplace=True)

    # 미확인 원소 대체
    train['시술 당시 나이'] = train['시술 당시 나이'].replace({'알 수 없음' : None})
    test['시술 당시 나이'] = test['시술 당시 나이'].replace({'알 수 없음' : None})
    train['특정 시술 유형'] = train['특정 시술 유형'].replace({'Unknown' : 'IVF'})
    test['특정 시술 유형'] = test['특정 시술 유형'].replace({'Unknown' : 'IVF'})
    train['난자 출처'] = train['난자 출처'].replace({'알 수 없음' : '본인 제공'})
    test['난자 출처'] = test['난자 출처'].replace({'알 수 없음' : '본인 제공'})

    # 결측치 최빈값 대체
    train = train.apply(lambda x:x.fillna(x.mode()[0]))
    test = test.apply(lambda x:x.fillna(x.mode()[0]))

    categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
    ]
    categorical_columns = [x for x in categorical_columns if x not in non_imp_cols]
    
    for col in categorical_columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)


    return train, test, categorical_columns

train, test, categorical_columns = preprocessing(train, test)

In [4]:
from sklearn.preprocessing import LabelEncoder

for col in categorical_columns:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']


In [5]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

from sklearn.preprocessing import PowerTransformer

numeric_cols = X.select_dtypes(include=[np.number]).columns

pt = PowerTransformer(method='yeo-johnson', standardize=True)
X[numeric_cols] = pt.fit_transform(X[numeric_cols])
test[numeric_cols] = pt.fit_transform(test[numeric_cols])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)



In [6]:
import pickle

# 'model.pkl' 파일에 저장된 모델 파라미터를 불러오기
with open('./best_params_cat_yj_drop.pkl', 'rb') as f:
    cat_params = pickle.load(f)

with open('./best_params_lightgbm_yj.pkl', 'rb') as f:
    lgbm_params = pickle.load(f)


In [ ]:

# 3. 학습/테스트 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 각 기본 모델 정의
# CatBoost: 범주형 변수는 이미 인코딩되어 있으므로 일반 모델로 사용
cat_model = CatBoostClassifier(**cat_params, random_state=42)
lgbm_model = LGBMClassifier(**lgbm_params, random_state=42)

meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 5. 스태킹 앙상블 모델 구성
# 기본 모델로 위의 세 모델을 사용하고, 최종 예측은 로지스틱 회귀로 결정
estimators = [
    ('cat', cat_model),
    ('lgbm', lgbm_model)
]
voting_clf = VotingClassifier(
    estimators=[
        ('cat', cat_model),
        ('lgbm', lgbm_model)
    ],
    voting='soft'  # 소프트 보팅: 각 모델의 예측 확률을 평균하여 최종 예측
)

# Voting 앙상블 모델 학습
voting_clf.fit(X_train, y_train)

# 6. 스태킹 모델 학습
voting_clf.fit(X_train, y_train)

# 7. 예측 및 평가
pred_probas = voting_clf.predict_proba(X_valid)[:, 1]
roc = roc_auc_score(y_valid, pred_probas)
print("Stacking 앙상블 모델 roc:", roc)

0:	learn: 0.6247445	total: 96.2ms	remaining: 42.2s
1:	learn: 0.5824391	total: 114ms	remaining: 24.9s
2:	learn: 0.5564588	total: 139ms	remaining: 20.2s
3:	learn: 0.5437123	total: 157ms	remaining: 17.1s
4:	learn: 0.5339971	total: 174ms	remaining: 15.1s
5:	learn: 0.5270136	total: 191ms	remaining: 13.8s
6:	learn: 0.5175160	total: 209ms	remaining: 12.9s
7:	learn: 0.5115835	total: 226ms	remaining: 12.2s
8:	learn: 0.5078071	total: 243ms	remaining: 11.6s
9:	learn: 0.5048760	total: 263ms	remaining: 11.3s
10:	learn: 0.5015119	total: 280ms	remaining: 10.9s
11:	learn: 0.4997643	total: 300ms	remaining: 10.7s
12:	learn: 0.4977010	total: 320ms	remaining: 10.5s
13:	learn: 0.4962445	total: 342ms	remaining: 10.4s
14:	learn: 0.4954220	total: 361ms	remaining: 10.2s
15:	learn: 0.4947922	total: 379ms	remaining: 10s
16:	learn: 0.4940983	total: 396ms	remaining: 9.85s
17:	learn: 0.4933215	total: 412ms	remaining: 9.67s
18:	learn: 0.4927708	total: 429ms	remaining: 9.51s
19:	learn: 0.4922651	total: 452ms	remainin

In [8]:
pred_probas = voting_clf.predict_proba(test)[:, 1]
print("예측 확률:", pred_probas)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_probas
sample_submission.to_csv('./submit/submit_40.csv', index=False)

예측 확률: [0.00162563 0.00105828 0.13737355 ... 0.34814185 0.15783065 0.00163645]
